```
The MIT License (MIT)

Copyright (c) 2020 NVIDIA

Permission is hereby granted, free of charge, to any person obtaining a copy of
this software and associated documentation files (the "Software"), to deal in
the Software without restriction, including without limitation the rights to
use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of
the Software, and to permit persons to whom the Software is furnished to do so,
subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS
FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR
COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER
IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN
CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

```

# Interacting with TRITON Inference Server

Walthrough a generic pipeline for:
- Using client libraries to run inference on models on TRITON sever
- Modifying model configs

#### Environment
**1 - Client**
All steps executed using **NGC Pytorch Docker (v 20.11)**
* TRITON Client libraries installed within container

**2 - Server**
using **NGC tritonserver Docker (20.11)**

![TRITON Workflow](./NB_images/TRITON_image.png)

Start a new terminal and start the TRITON server by running the following command

`start_triton_server.sh`

This simulates a "Server" running in a separate docker container

In [1]:
import argparse
import numpy as np
import os
import json
from builtins import range

from lib.dataset_utils import *
from lib.triton_utils import *

import tritonclient.grpc.model_config_pb2 as mc
import tritonclient.http as httpclient
from tritonclient.utils import triton_to_np_dtype
from tritonclient.utils import InferenceServerException

In [2]:
list_files('./trtismodelrepo/v1/model_cxr_trt/')

/
    config.pbtxt
    labels.txt
1/
    model.plan


In [3]:
!cat './trtismodelrepo/v1/model_cxr_trt/config.pbtxt'

name: "model_cxr_trt"
platform: "tensorrt_plan"
max_batch_size : 96
input [
  {
    name: "input"
    data_type: TYPE_FP32
    format: FORMAT_NCHW
    dims: [ 3, 224,224 ]
  }
]
output [
  {
    name: "output"
    data_type: TYPE_FP32
    dims: [ 14 ]
    label_filename: "labels.txt"
  }
]

dynamic_batching {
  preferred_batch_size: [ 80 ]
}



## Check status of Inference Server/Specific Model

In [13]:
# !curl localhost:8000/api/status

In [24]:
triton_client = httpclient.InferenceServerClient(url='localhost:8000')

#Health Check
try:
    metadata = triton_client.get_server_metadata()
except InferenceServerException as e:
        print("failed to retrieve the metadata: " + str(e))
# if not (metadata['name'] == 'triton'):
#     print("FAILED : get_server_metadata")
# #         sys.exit(1)
print(metadata)

{'name': 'triton', 'version': '2.5.0', 'extensions': ['classification', 'sequence', 'model_repository', 'schedule_policy', 'model_configuration', 'system_shared_memory', 'cuda_shared_memory', 'binary_tensor_data', 'statistics']}


In [29]:
model_name = "model_cxr_trt"
# Get model status on server
try:
    model_metadata = triton_client.get_model_metadata(model_name)
except InferenceServerException as e:
        print("failed to retrieve the metadata: " + str(e))

print(model_metadata)

{'name': 'model_cxr_trt', 'versions': ['1'], 'platform': 'tensorrt_plan', 'inputs': [{'name': 'input', 'datatype': 'FP32', 'shape': [-1, 3, 224, 224]}], 'outputs': [{'name': 'output', 'datatype': 'FP32', 'shape': [-1, 14]}]}


In [31]:
try:
    model_config = triton_client.get_model_config(model_name)
except InferenceServerException as e:
        print("failed to retrieve the config: " + str(e))

print(model_config)

{'name': 'model_cxr_trt', 'platform': 'tensorrt_plan', 'backend': '', 'version_policy': {'latest': {'num_versions': 1}}, 'max_batch_size': 96, 'input': [{'name': 'input', 'data_type': 'TYPE_FP32', 'format': 'FORMAT_NCHW', 'dims': [3, 224, 224], 'is_shape_tensor': False, 'allow_ragged_batch': False}], 'output': [{'name': 'output', 'data_type': 'TYPE_FP32', 'dims': [14], 'label_filename': 'labels.txt', 'is_shape_tensor': False}], 'batch_input': [], 'batch_output': [], 'optimization': {'priority': 'PRIORITY_DEFAULT', 'input_pinned_memory': {'enable': True}, 'output_pinned_memory': {'enable': True}}, 'dynamic_batching': {'preferred_batch_size': [80], 'max_queue_delay_microseconds': 0, 'preserve_ordering': False, 'priority_levels': 0, 'default_priority_level': 0, 'priority_queue_policy': {}}, 'instance_group': [{'name': 'model_cxr_trt', 'kind': 'KIND_GPU', 'count': 1, 'gpus': [0], 'profile': []}], 'default_model_filename': 'model.plan', 'cc_model_filenames': {}, 'metric_tags': {}, 'paramete

In [41]:
def parse_model(model_metadata, model_config):
    """
    Check the configuration of a model to make sure it meets the
    requirements
    """
    if len(model_metadata['inputs']) != 1:
        raise Exception("expecting 1 input, got {}".format(
            len(model_metadata['inputs'])))
    if len(model_metadata['outputs']) != 1:
        raise Exception("expecting 1 output, got {}".format(
            len(model_metadata['outputs'])))

    if len(model_config['input']) != 1:
        raise Exception(
            "expecting 1 input in model configuration, got {}".format(
                len(model_config['input'])))

    input_metadata = model_metadata['inputs'][0]
    input_config = model_config['input'][0]
    output_metadata = model_metadata['outputs'][0]
    
    max_batch_size = 0
    if 'max_batch_size' in model_config:
        max_batch_size = model_config['max_batch_size']
        
    # Output is expected to be a vector. But allow any number of
    # dimensions as long as all but 1 is size 1 (e.g. { 10 }, { 1, 10
    # }, { 10, 1, 1 } are all ok). Ignore the batch dimension if there
    # is one.
    output_batch_dim = (max_batch_size > 0)
    non_one_cnt = 0
    for dim in output_metadata['shape']:
        if output_batch_dim:
            output_batch_dim = False
        elif dim > 1:
            non_one_cnt += 1
            if non_one_cnt > 1:
                raise Exception("expecting model output to be a vector")
    # Model input must have 3 dims (not counting the batch dimension),
    # either CHW or HWC
    input_batch_dim = (max_batch_size > 0)
    expected_input_dims = 3 + (1 if input_batch_dim else 0)
    if len(input_metadata['shape']) != expected_input_dims:
        raise Exception(
            "expecting input to have {} dimensions, model '{}' input has {}".
            format(expected_input_dims, model_metadata['name'],
                   len(input_metadata['shape'])))

    if input_config['format'] == "FORMAT_NHWC":
        h = input_metadata['shape'][1 if input_batch_dim else 0]
        w = input_metadata['shape'][2 if input_batch_dim else 1]
        c = input_metadata['shape'][3 if input_batch_dim else 2]
    else:
        c = input_metadata['shape'][1 if input_batch_dim else 0]
        h = input_metadata['shape'][2 if input_batch_dim else 1]
        w = input_metadata['shape'][3 if input_batch_dim else 2]
    
    return (max_batch_size, input_metadata['name'], output_metadata['name'], c,
            h, w, input_config['format'], input_metadata['datatype'])

In [38]:
max_batch_size, input_name, output_name, c, h, w, format, dtype = parse_model(model_metadata,model_config)

## Setup Data for inference

In [7]:
from lib.dataset_utils import *

def get_batched_image_input(bs):
    all_inputs = []
    dataset_json = 'CXR14_dataset.json'
    with open(dataset_json) as json_file:
        data = json.load(json_file)
    
    test_loader = get_test_loader_TTA(data, batch_size=bs)

    # Using Test Time Augmentation    
    for i, (input, target) in enumerate(test_loader):
        bs, n_crops, c, h, w = input.size()
        input = input.view(-1, c, h, w)

        all_inputs.append(input)

    return all_inputs, data['labels'][0]

bs = 8 
im_tensor, labels = get_batched_image_input(bs)

## Setup inference on images

In [42]:
def triton_inferer(input_data, model_name, input_name, output_name, headers=None):
    inputs = []
    outputs = []
    inputs.append(httpclient.InferInput(input_name, [80, 3, 224, 224], "FP32"))
#     inputs.append(httpclient.InferInput('INPUT1', [1, 16], "INT32"))

    # Initialize the data
    inputs[0].set_data_from_numpy(input_data.cpu().numpy(), binary_data=True)
#     inputs[1].set_data_from_numpy(input1_data, binary_data=True)

    query_params = {'test_1': 1, 'test_2': 2}
    results = triton_client.infer(model_name,
                                  inputs,
                                  outputs=None,
#                                   query_params=query_params,
                                  headers=headers)

    return results.as_numpy(output_name)

def process_results_cxr(out):
    outputs = torch.cat([torch.tensor(np.atleast_2d(im)) for im in out])
    output = torch.nn.functional.softmax(outputs, dim=1).cpu()
    output = output.view(8, 10, -1).mean(1).numpy()
    top5 = np.argsort(output)[:,-5:][:,::-1]


    for i in range(8):
        t5 = top5[i]
        op = output[i]
        txt = 'Image {} -- '.format(i+1)
        for c, v in zip(t5, op[t5]):
            txt = txt + f"{labels[str(c)]}: {100*v:.1f}% "
        print(txt)

In [43]:
model_name = "model_cxr_trt"
out = triton_inferer(im_tensor[0], model_name,input_name, output_name)
process_results_cxr(out)

Image 1 -- Cardiomegaly: 90.4% Effusion: 5.7% Emphysema: 2.0% Nodule: 0.5% Pleural_Thickening: 0.5% 
Image 2 -- Cardiomegaly: 91.4% Emphysema: 3.2% Effusion: 2.7% Infiltration: 1.0% Nodule: 0.5% 
Image 3 -- Cardiomegaly: 88.1% Effusion: 7.4% Emphysema: 1.6% Infiltration: 0.8% Atelectasis: 0.6% 
Image 4 -- Infiltration: 33.5% Nodule: 32.7% Emphysema: 9.1% Effusion: 8.8% Pleural_Thickening: 5.6% 
Image 5 -- Nodule: 51.6% Mass: 40.5% Infiltration: 4.2% Consolidation: 1.7% Atelectasis: 1.4% 
Image 6 -- Infiltration: 22.7% Nodule: 22.0% Atelectasis: 19.1% Pleural_Thickening: 8.7% Effusion: 6.9% 
Image 7 -- Infiltration: 39.7% Nodule: 30.7% Atelectasis: 7.3% Consolidation: 6.3% Pleural_Thickening: 4.5% 
Image 8 -- Infiltration: 34.8% Pneumothorax: 25.8% Atelectasis: 16.2% Consolidation: 9.7% Nodule: 3.9% 


In [44]:
model_name = "model_cxr_trt_fp16"
out = triton_inferer(im_tensor[0], model_name,input_name, output_name)
process_results_cxr(out)

Image 1 -- Cardiomegaly: 90.3% Effusion: 5.7% Emphysema: 2.0% Nodule: 0.5% Pleural_Thickening: 0.5% 
Image 2 -- Cardiomegaly: 91.3% Emphysema: 3.2% Effusion: 2.8% Infiltration: 1.0% Nodule: 0.5% 
Image 3 -- Cardiomegaly: 88.0% Effusion: 7.5% Emphysema: 1.6% Infiltration: 0.8% Atelectasis: 0.7% 
Image 4 -- Infiltration: 33.6% Nodule: 32.6% Emphysema: 9.0% Effusion: 8.9% Pleural_Thickening: 5.6% 
Image 5 -- Nodule: 51.5% Mass: 40.7% Infiltration: 4.1% Consolidation: 1.7% Atelectasis: 1.4% 
Image 6 -- Infiltration: 22.7% Nodule: 22.1% Atelectasis: 18.9% Pleural_Thickening: 8.7% Effusion: 6.8% 
Image 7 -- Infiltration: 39.6% Nodule: 30.8% Atelectasis: 7.2% Consolidation: 6.3% Pleural_Thickening: 4.5% 
Image 8 -- Infiltration: 34.7% Pneumothorax: 26.1% Atelectasis: 16.1% Consolidation: 9.7% Nodule: 3.9% 


In [45]:
model_name = "model_cxr_onnx"
out = triton_inferer(im_tensor[0], model_name,input_name, output_name)
process_results_cxr(out)

Image 1 -- Cardiomegaly: 90.4% Effusion: 5.7% Emphysema: 2.0% Nodule: 0.5% Pleural_Thickening: 0.5% 
Image 2 -- Cardiomegaly: 91.4% Emphysema: 3.2% Effusion: 2.7% Infiltration: 1.0% Nodule: 0.5% 
Image 3 -- Cardiomegaly: 88.1% Effusion: 7.4% Emphysema: 1.6% Infiltration: 0.8% Atelectasis: 0.6% 
Image 4 -- Infiltration: 33.5% Nodule: 32.7% Emphysema: 9.1% Effusion: 8.8% Pleural_Thickening: 5.6% 
Image 5 -- Nodule: 51.6% Mass: 40.5% Infiltration: 4.2% Consolidation: 1.7% Atelectasis: 1.4% 
Image 6 -- Infiltration: 22.7% Nodule: 22.0% Atelectasis: 19.1% Pleural_Thickening: 8.7% Effusion: 6.9% 
Image 7 -- Infiltration: 39.7% Nodule: 30.7% Atelectasis: 7.3% Consolidation: 6.3% Pleural_Thickening: 4.5% 
Image 8 -- Infiltration: 34.8% Pneumothorax: 25.8% Atelectasis: 16.2% Consolidation: 9.7% Nodule: 3.9% 


In [20]:
model_name = "model_cxr_onnx"
# Get model status on server
metadata = triton_client.get_model_metadata(model_name,
                                                query_params={
                                                    'test_1': 1,
                                                    'test_2': 2
                                                })
if not (metadata['name'] == model_name):
    print("FAILED : get_model_metadata")
print(metadata)

{'name': 'model_cxr_onnx', 'versions': ['1'], 'platform': 'onnxruntime_onnx', 'inputs': [{'name': 'input', 'datatype': 'FP32', 'shape': [-1, 3, 224, 224]}], 'outputs': [{'name': 'output', 'datatype': 'FP32', 'shape': [-1, 14]}]}


In [22]:
import time

In [23]:
counter = 1
while counter < 10:
    out = triton_inferer(im_tensor[0], model_name)
    if counter % 100 == 0: print('Iteration {}'.format(counter))
    time.sleep(2)
    counter += 1